### Plan of the analysis
1. Map transcript sequence with BLAST
2. Conver to BED format to visualise in UCSC Genome Browser
3. For each hit define cytological band and censat annotation ID of sattelite

In [42]:
import os
from collections import defaultdict

## input_files

transcript = "./transcript_4_1.fasta"
human_t2t_genome = "./human_t2t_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

## output files 
output_folder = "./result"
blast_result = "./result/trascript_to_t2t.outfmt6"
blast_result_bed = f"{blast_result}.bed"
blast_results_with_annotation = "./result/transcript_to_t2t_with_censat_and_locus.tsv"
blast_results_with_annotation_summary = "./result/transcript_to_t2t_with_censat_and_locus_summary.tsv"

In [18]:
## Download human T2T genome

if not os.path.exists("./human_t2t_genome"):
    os.makedirs("./human_t2t_genome")

if not os.path.exists(human_t2t_genome):
    os.system(f"wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/914/755/GCF_009914755.1_T2T-CHM13v2.0/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna.gz -O {human_t2t_genome}")
    os.system(f"gunzip ./human_t2t_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna.gz")
    print('Download of the genome done!')

In [20]:
## Create index of T2T assembly 

if not os.path.exists(human_t2t_genome + ".ndb"):
    os.system(f'makeblastdb -dbtype nucl -in {human_t2t_genome}')

In [21]:
## Map transcript to T2T assembly

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(blast_result):
    os.system(f"blastn -db {human_t2t_genome} -query {transcript} -outfmt 6 -out {blast_result} -task blastn -perc_identity 80")

In [24]:
## Convert outfmt6 to BED

if not os.path.exists("./blast2bed"):
    os.system("git clone git@github.com:nterhoeven/blast2bed.git")

if not os.path.exists(blast_result_bed):
    os.system(f"./blast2bed/blast2bed {blast_result}")

In [39]:
## Functions to work with TSV tables

def read_tsv_to_list(tsv):
    result = []
    with open(tsv) as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            line = line.strip().split("\t")
            result.append(line)
    return result

def read_tsv_to_dict(tsv, key_index):
    result = {}
    with open(tsv) as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            line = line.strip().split("\t")
            result[line[key_index]] = line
    return result

In [28]:
## Parse annotations

sat_t2t_coordinates = "./ucsc_data/censat.bed"
locus_t2t_coodinates = "./ucsc_data/cytoBandMapped.bed"
chr_alias = "./ucsc_data/chr_alias.txt"


sat_t2t = read_tsv_to_list(sat_t2t_coordinates)
locus_t2t = read_tsv_to_list(locus_t2t_coodinates)
alias_refseq = read_tsv_to_dict(chr_alias, 2)

In [29]:
blast_run_t41_bed = read_tsv_to_list(blast_result_bed)

In [30]:
## Add censat ID and locus to each blast hit

blast_with_censat_locus = []
for match in blast_run_t41_bed[1:]:
    new_match = match
    new_match.append("")
    new_match.append("")
    refseq_chr = match[0]
    new_match[0] = alias_refseq[refseq_chr][0]
    match_chrom = new_match[0]
    start = int(match[1])
    stop = int(match[2])
    for sat in sat_t2t:
        sat_chrom = sat[0]
        sat_start = int(sat[1])
        sat_stop = int(sat[2])
        if sat_chrom == match_chrom:
            if (start >= sat_start and start <= sat_stop) and (stop <= sat_stop and stop >= sat_start):
                match[-2] = sat[3]
    for locus in locus_t2t:
        locus_chrom = locus[0]
        locus_start = int(locus[1])
        locus_stop = int(locus[2])
        if locus_chrom == match_chrom:
            if (start >= locus_start and start <= locus_stop) and (stop <= locus_stop and stop >= locus_start):
                match[-1] = locus[3]
    blast_with_censat_locus.append(match)

In [43]:
## Write annotated hits table

with open(blast_results_with_annotation, "w") as fw:
    fw.write('Chromosome	Start	Stop	Target	Strand	Sattelite	Locus\n')
    for line in blast_with_censat_locus:
        line_to_write = "\t".join(line) + "\n"
        fw.write(line_to_write)

In [33]:
## Sreate summary for each chromosome

summary_by_chr_blast = defaultdict(list)
for i in blast_with_censat_locus:
    chrom = i[0]
    sat = i[-2]
    locus = i[-1]
    if len(summary_by_chr_blast[chrom]) != 2:
        summary_by_chr_blast[chrom].append({})
        summary_by_chr_blast[chrom].append([])
    if not sat in summary_by_chr_blast[chrom][0].keys():
        if sat:
            summary_by_chr_blast[chrom][0][sat] = [0, locus]
    else:
        summary_by_chr_blast[chrom][0][sat][0] += 1

In [46]:
with open(blast_results_with_annotation_summary, "w") as fw:
    fw.write('Chromosome	Sattelite	N_hits	Locus\n')
    for chrom, summary in summary_by_chr_blast.items():
        for sat, info in summary[0].items():
            number = info[0]
            locus = info[1]
            line_to_write = "\t".join([chrom, sat, str(number), locus+"\n"])
            fw.write(line_to_write)